In [ ]:
import numpy as np
import pandas as pd
import schedule
import time # to make system sleep after avery call.
import requests
from datetime import datetime

import requests

import sched
s = sched.scheduler(time.time, time.sleep)

def apicall():  
    now = datetime.now()
    mtm = now.strftime("%H:%M:%S")
    response = requests.get('https://iot-rest-prod.urbani.se/api/v1/customer/S_Tu2CLJlIFptGnSDX0Yew/alerts?size=200&interval=month', headers={'accept': 'application/json', 'X-User-Email': 'amughrabi@alfoadia.com.sa', 'X-User-Token': 'AChQZ9UWDyDe1W8WZ9QU'})

    print(response.status_code)
    print(response.json())

    import json

    def jprint(obj):
        # create a formatted string of the Python JSON object
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)

    jprint(response.json())

    raw_json = response.json()

    df = pd.DataFrame(raw_json["alerts"])
    df

    df["ref"][0]
    data_list = df["ref"].tolist()
    data_list

    df_ref = pd.DataFrame(data_list)
    df_ref = df_ref[["c", "d", "ds", "l", "m", "o", "unit"]]
    df_ref

    data_list1 = df["threshold"].tolist()
    data_list1

    df["threshold"][0]

    df1 = pd.DataFrame(df["threshold"])
    df1

    df1['threshold'] = df1['threshold'].astype(str)

    new  = df1['threshold'].str.split(",", n = 4, expand = True) 
    print(new)
    print(new[0])
    print(new[1])
    print(new[2])
    print(new[3])

    #Label
    new[0] = new[0].astype(str).str.replace("{","").str.replace("'","")
    new[0] = new[0].astype(str).str.replace("label: ","")
    new[0]

    #Range From
    new[1] = new[1].astype(str).str.replace("{","").str.replace("'","")
    new[1] = new[1].astype(str).str.replace("range: from: ","")
    new[1] = new[1].astype(str).str.replace("status: ","")
    new[1] 

    #Range to
    new[2] = new[2].astype(str).str.replace("}","").str.replace("'","")
    new[2] = new[2].astype(str).str.replace("to: ","")
    new[2]

    #Status
    new[3] = new[3].astype(str).str.replace("}","").str.replace("'","")
    new[3] = new[3].astype(str).str.replace("status: ","")
    new[3]

    new
    
    new[3][new[1] == ' danger'] = ' danger'
    new[3][new[1] == ' warning'] = ' warning'
    new[3]
    
    new[1][new[1] == ' danger'] = new[2]
    new[1][new[1] == ' warning'] = new[2]
    new[1]
    
    new[2][new[1] == new[2]] = ''
    new[2]
    
    new
    
    new[1] = new[1].astype(str).str.replace(" value:","")
    new
    

    df_thres = new.rename(columns={ 0:"label", 1: "range_from", 2: "range_to", 3: "status"})
    df_thres

    df_ref_thres = pd.concat([df_ref, df_thres], axis=1)
    df_ref_thres

    df_data = pd.concat([df, df_ref_thres], axis=1)
    df_data

    df_data = df_data.drop(['ref', 'threshold', 'triggered'], axis=1)
    df_data

    df_data1 = df_data.rename(columns={"c": "ref_c", "d": "ref_d", "ds": "ref_ds", "l": "ref_l", "m": "ref_m", "o": "ref_o",\
                                "unit": "ref_unit", "label": "thres_label", "range_from": "thres_range_from",\
                                "range_to": "thres_range_to", "status": "thres_status"})
    df_data1

    df_final =df_data1.rename(columns={"di": "Device ID"})
    df_final
    
    #df_final = pd.DataFrame(columns=df_final.columns)
    #df_final
    
    def convertTime(t):
        t = int(t)
        return datetime.fromtimestamp(t)

    df_final = df_final.dropna(subset=["ts"])
    df_final["Device ID"].fillna(value="", inplace=True)

    df_final["ts"]= df_final["ts"]/1000
    df_final["ts"] = df_final["ts"].apply(convertTime)

    df_final.head()

    def convertTime(t):
        t = int(t)
        return datetime.fromtimestamp(t)

    df_final = df_final.dropna(subset=["tts"])
    df_final["Device ID"].fillna(value="", inplace=True)

    df_final["tts"]= df_final["tts"]/1000
    df_final["tts"] = df_final["tts"].apply(convertTime)

    df_final.head()

    df_loc = pd.read_csv ('Alert_Updated_SMS_DISTRIBUTION_SHEET.csv')
    df_loc

    df_loc.shape

    df_final.shape

    print(df_loc.info())

    print(df_final.info())

    df_final['Device ID'] = df_final['Device ID'].astype(int)

    df_loc ['Device ID']= df_loc ['Device ID'].astype(int)

    print(df_final.info())

    print(df_loc.info())

    result = pd.merge(df_loc, df_final, how ='inner', on ='Device ID')
    result

    result = result.replace('nan', float('nan'))
    result

    print(result.info())

    result.columns


    #locations 
    top_location = result['Location'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df1 = pd.DataFrame(top_location.index)
    loc = top_location.reset_index(drop=True)
    loc.columns = ['1']
    loc = pd.merge(left=df1, right=loc, left_on=df1.index, right_on=loc.index)
    toplocation = loc.drop(['key_0'], axis=1)
    toplocation = toplocation.rename(columns={ 0:"Locations", "Location": "Location_Count"})
    toplocation = toplocation.sort_values(by = 'Location_Count',ascending=False)
    toplocation




    #Device Name 
    top_devices = result['device Name'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df2 = pd.DataFrame(top_devices.index)
    devi = top_devices.reset_index(drop=True)
    devi.columns = ['1']
    devi = pd.merge(left=df2, right=devi, left_on=df2.index, right_on=devi.index)
    topdevices = devi.drop(['key_0'], axis=1)
    topdevices = topdevices.rename(columns={ 0:"Devices", "device Name": "Device_Count"})
    topdevices = topdevices.sort_values(by = 'Device_Count',ascending=False)
    topdevices





    #Device IDs 
    top_deviceIDs = result['Device ID'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df3 = pd.DataFrame(top_deviceIDs.index)
    devids = top_deviceIDs.reset_index(drop=True)
    devids.columns = ['1']
    devids = pd.merge(left=df3, right=devids, left_on=df3.index, right_on=devids.index)
    topdeviceIDs = devids.drop(['key_0'], axis=1)
    topdeviceIDs = topdeviceIDs.rename(columns={ 0:"Device_IDs", "Device ID": "DeviceID_Count"})
    topdeviceIDs = topdeviceIDs.sort_values(by = 'DeviceID_Count',ascending=False)
    topdeviceIDs

    SameDev = topdeviceIDs[topdeviceIDs['DeviceID_Count'] != 1]
    print(SameDev)
    
    SumSameDev = SameDev['DeviceID_Count'].sum(axis=0) 
    print(SumSameDev)


    #Device State 
    top_deviceSta = result['state'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df4 = pd.DataFrame(top_deviceSta.index)
    devist = top_deviceSta.reset_index(drop=True)
    devist.columns = ['1']
    devist = pd.merge(left=df4, right=devist, left_on=df4.index, right_on=devist.index)
    topdeviceSta = devist.drop(['key_0'], axis=1)
    topdeviceSta = topdeviceSta.rename(columns={ 0:"Device_State", "state": "State_Count"})
    topdeviceSta = topdeviceSta.sort_values(by = 'State_Count',ascending=False)
    topdeviceSta





    #Device Type 
    top_deviceTyp = result['type'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df5 = pd.DataFrame(top_deviceTyp.index)
    devityp = top_deviceTyp.reset_index(drop=True)
    devityp.columns = ['1']
    devityp = pd.merge(left=df5, right=devityp, left_on=df5.index, right_on=devityp.index)
    topdeviceTyp = devityp.drop(['key_0'], axis=1)
    topdeviceTyp = topdeviceTyp.rename(columns={ 0:"Device_Type", "type": "Type_Count"})
    topdeviceTyp = topdeviceTyp.sort_values(by = 'Type_Count',ascending=False)
    topdeviceTyp





    #Device Reference Measure
    top_deviceM = result['ref_m'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df6 = pd.DataFrame(top_deviceM.index)
    devim = top_deviceM.reset_index(drop=True)
    devim.columns = ['1']
    devim = pd.merge(left=df6, right=devim, left_on=df6.index, right_on=devim.index)
    topdeviceM = devim.drop(['key_0'], axis=1)
    topdeviceM = topdeviceM.rename(columns={ 0:"Reference_Measure", "ref_m": "Measure_Count"})
    topdeviceM = topdeviceM.sort_values(by = 'Measure_Count',ascending=False)
    topdeviceM




    #Device Reference Unit
    top_deviceU = result['ref_unit'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df7 = pd.DataFrame(top_deviceU.index)
    deviu = top_deviceU.reset_index(drop=True)
    deviu.columns = ['1']
    deviu = pd.merge(left=df7, right=deviu, left_on=df7.index, right_on=deviu.index)
    topdeviceU = deviu.drop(['key_0'], axis=1)
    topdeviceU = topdeviceU.rename(columns={ 0:"Reference_Unit", "ref_unit": "Unit_Count"})
    topdeviceU = topdeviceU.sort_values(by = 'Unit_Count',ascending=False)
    topdeviceU





    #Device Threshold Label
    top_deviceL = result['thres_label'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df8 = pd.DataFrame(top_deviceL.index)
    devil = top_deviceL.reset_index(drop=True)
    devil.columns = ['1']
    devil = pd.merge(left=df8, right=devil, left_on=df8.index, right_on=devil.index)
    topdeviceL = devil.drop(['key_0'], axis=1)
    topdeviceL = topdeviceL.rename(columns={ 0:"Threshold_Label", "thres_label": "Label_Count"})
    topdeviceL = topdeviceL.sort_values(by = 'Label_Count',ascending=False)
    topdeviceL





    #Device Threshold Status
    top_deviceS = result['thres_status'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    df9 = pd.DataFrame(top_deviceS.index)
    devis = top_deviceS.reset_index(drop=True)
    devis.columns = ['1']
    devis = pd.merge(left=df9, right=devis, left_on=df9.index, right_on=devis.index)
    topdeviceS = devis.drop(['key_0'], axis=1)
    topdeviceS = topdeviceS.rename(columns={ 0:"Threshold_Status", "thres_status": "Status_Count"})
    topdeviceS = topdeviceS.sort_values(by = 'Status_Count',ascending=False)
    topdeviceS






    #Location Count
    location_count = df_loc['Location'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    dfLoc = pd.DataFrame(location_count.index)
    loc_c = location_count.reset_index(drop=True)
    loc_c.columns = ['1']
    loc_c = pd.merge(left=dfLoc, right=loc_c, left_on=dfLoc.index, right_on=loc_c.index)
    locationcount = loc_c.drop(['key_0'], axis=1)
    locationcount = locationcount.rename(columns={ 0:"Top_Locations", "Location": "Location_Count"}).head(10)
    locationcount = locationcount.sort_values(by = 'Location_Count',ascending=False)
    locationcount

    #Same Location of Devices (coordiantes) 
    Same_Lat = result['Latitude'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    dfLat = pd.DataFrame(Same_Lat.index)
    deviLat = Same_Lat.reset_index(drop=True)
    deviLat.columns = ['1']
    deviLat = pd.merge(left=dfLat, right=deviLat, left_on=dfLat.index, right_on=deviLat.index)
    SameLat = deviLat.drop(['key_0'], axis=1)
    SameLat = SameLat.rename(columns={ 0:"Latitude", "Latitude": "Same_Lat_Count"})
    SameLat = SameLat.sort_values(by = 'Same_Lat_Count',ascending=False)
    SameLat

    
    #Same Location of Devices (coordiantes) 
    Same_Long = result['Longitude'].value_counts()
    #Convert data to dataframe
    #Remove Unknown location
    dfLong = pd.DataFrame(Same_Long.index)
    deviLong = Same_Long.reset_index(drop=True)
    deviLong.columns = ['1']
    deviLong = pd.merge(left=dfLong, right=deviLong, left_on=dfLong.index, right_on=deviLong.index)
    SameLong = deviLong.drop(['key_0'], axis=1)
    SameLong = SameLong.rename(columns={ 0:"Longitude", "Longitude": "Same_Long_Count"})
    SameLong = SameLong.sort_values(by = 'Same_Long_Count',ascending=False)
    SameLong

    SameLat = SameLat[SameLat['Same_Lat_Count'] != 1]
    print(SameLat)
    SameLong = SameLong[SameLong['Same_Long_Count'] != 1]
    print(SameLong)

    
    SumSameLat = SameLat['Same_Lat_Count'].sum(axis=0) 
    print(SumSameLat)
    SumSameLong = SameLong['Same_Long_Count'].sum(axis=0) 
    print(SumSameLong)

    if SumSameLat == SumSameLong:
        SameCoordinates = SumSameLat
        print(SameCoordinates)
    else:
        SameCoordinates = SumSameLat
        print(SameCoordinates)
    
    #count
    #total_topdeviceTyp=topdeviceTyp.iloc[0, 1]
    total_topdeviceTyp=result.shape[0]
    total_topdeviceTyp


    total_SameDev=SumSameDev
    total_SameDev
    
    total_SameLat = SameCoordinates
    total_SameLat


    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots





    fig = make_subplots(
    
    rows = 4, cols = 6,
    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
    )


    #result['thres_status'] = result['thres_status'].replace(float('nan'), 'normal')
    #result['thres_status'] = result['thres_status'].replace('nan', 'danger')
    #result['thres_status'][0] = 'danger'
    result['thres_status']
    
    table1 = result[['Device ID', 'device Name', 'Location', 'Latitude','Longitude', 'state', 'ts', 'type', 'v',\
                'ref_m', 'ref_unit','thres_status']]
    table1 = table1.sort_values(by=['ts'], ascending=False)
    
    table1.insert(0, 'ID', range(1, 1 + len(table1)))
    #table1.ID[1] = mtm

    table1

    message = "Device ID: " + result["Device ID"].astype(str) + "<br>"
    message += "Location: " + result["Location"].astype(str) + "<br>"
    message += "Device Name: " + result["device Name"].astype(str) + "<br>"
    message += "Type: " + result["type"].astype(str) + "<br>"
    message += "State: " + result["state"].astype(str) + "<br>"
    message += "Label: " + result["thres_label"].astype(str) + "<br>"
    message += "Threshold Value/Range: " + result["thres_range_from"].astype(str)+" - "+result["thres_range_to"].astype(str) + "<br>"
    message += "Threshold status: " + result["thres_status"].astype(str) + "<br>"
    message += "Unit: " + result["ref_unit"].astype(str) + "<br>"
    message += "V: " + result["v"].astype(str) + "<br>"
    message += "Last updated: " + result["ts"].astype(str)
    result["text"] = message
    

    #Create New Column based on previous column values
    status_count1 = np.where(result['thres_status'] == ' danger',1,\
                                       np.where(result['thres_status'] == ' warning',2,3))
    status_count1
    
    

    #Create New Column based on previous column values
    def f(row):
        if row['thres_status'] == ' warning':
            val = 1
        elif row['thres_status'] == ' danger':
            val = 2
        else: 
            val = 0

        return val

    status_count2 = result.apply(f, axis=1)
    status_count2
    
    

    color1 = result['thres_status'].str.replace(" ","").unique().astype(str)
    color1  
    
    
    
    col = pd.DataFrame(color1)
    col
    
    if 'nan' in color1:
        A = 1
    else:
        A = 0
    A
    
    if 'warning' in color1:
        B = 1
    else:
        B = 0
    B
    
    if 'danger' in color1:
        C = 1
    else:
        C = 0
    C
    
    color_names = col
    color_vals = list(range(len(color_names)))
    num_colors = len(color_vals)
    
    cmin = -0.5
    cmax = num_colors - 0.5
    
    if num_colors == 3:
        colorscale = [
        [0, 'rgb(255,0,0)'], #Normal
        #[0.33, 'rgb(255,0,0)'],
        #[0.33, 'rgb(255,255,0'], #Warning
        [0.5, 'rgb(255,255,0)'],
        #[0.66, 'rgb(255,0,0)'], #Danger
        [1, 'rgb(255,0,0)']]
        col1 = col.reindex([0, 2, 1])
        col1 = col1.reset_index(drop=True)
        #cmin = 0
        #cmax = num_colors 
        result['status_count'] = np.where(result['thres_status'] == ' danger',0,\
                                       np.where(result['thres_status'] == ' warning',1,\
                                                np.where(result['thres_status'] == 'normal',2,3)))

    elif num_colors == 2:
        if  A == 1 & B == 1: #normal and warning
            colorscale = [
            [0, 'rgb(255,0,0)'], #Normal
            [0.5, 'rgb(255,0,0)'],
            [0.5, 'rgb(255,255,0)'], #Warning
            [1, 'rgb(255,255,0)']]
            col1 = col.reindex([0, 1])
            col1 = col1.reset_index(drop=True)
            result['status_count'] = np.where(result['thres_status'] == ' warning',0,\
                                       np.where(result['thres_status'] == 'normal',1,3))

        elif A == 1 & C == 1: #normal and danger
            colorscale = [
            [0, 'rgb(255,0,0)'], #Normal
            [0.5, 'rgb(255,0,0)'],
            [0.5, 'rgb(255,0,0)'], #Danger
            [1, 'rgb(255,0,0)']]
            col1 = col.reindex([1, 0])
            col1 = col1.reset_index(drop=True)
            result['status_count'] = np.where(result['thres_status'] == ' danger',0,\
                                       np.where(result['thres_status'] == 'normal',1,3))

        elif B == 1 & C == 1: #warning and danger
            colorscale = [
            [0,'rgb(255,255,0)'], #Warning
            [0.5, 'rgb(255,255,0)'],
            [0.5, 'rgb(255,0,0)'], #Danger
            [1, 'rgb(255,0,0)']]
            col1 = col.reindex([0, 1])
            col1 = col1.reset_index(drop=True)
            result['status_count'] = np.where(result['thres_status'] == ' danger',0,\
                                       np.where(result['thres_status'] == ' warning',1,3))
            
    elif num_colors == 1:
        col1 = col
        result['status_count'] = 0
        if result['thres_status'][0] == ' danger':
            colorscale = [[0, 'rgb(255,0,0)'], [1, 'rgb(255,0,0)']]
        elif result['thres_status'][0] == ' warning':
            colorscale = [[0, 'rgb(255,255,0)'], [1, 'rgb(255,255,0)']]
        else:
            colorscale = [[0, 'rgb(255,255,255)'], [1, 'rgb(255,255,255)']]

    else:
        col1 = col
        colorscale = 'aggrnyl'
        result['status_count'] = float('nan')
     
    
    color_names = col1
    color_vals = list(color_names.index)
    num_colors = len(color_vals)
    
    device_name = np.where(result["Location"].str.startswith('HQ') == True,'HQ',result["Location"])

    fig.add_trace(go.Scattergeo(

            locationmode = "ISO-3",
            lon = result["Longitude"],
            lat = result["Latitude"],
            text = device_name,
            textposition="top center",
            hovertext = result["text"],
            hoverinfo = 'text',
            textfont = {"color": 'black',
                    "family":'Times New Roman',
                    "size":8},
            showlegend=False,
            mode = 'markers+text',
            marker = dict(
                size = 8,
                opacity = 0.8,
                reversescale = True,
                autocolorscale = False,
                symbol = 'square',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'
                ),
                colorbar = {'tickvals': color_vals, 'ticktext': color_names},
                colorscale = colorscale,
                cmin = cmin,
                cmax = cmax,
                color = result['status_count'],
                colorbar_title="Alerts<br>Threshold Status",
                colorbar_x = -0.05,
                #colorbar_len = 1
                
            )),
        row=1, col=1
    )


    fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_topdeviceTyp,
        title="Total Devices",
    ),
    row=1, col=4
    )

    fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_SameDev,
        title="Same Device IDs",
    ),
    row=1, col=5
    )

    fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_SameLat,
        title="Same Locations",
    ),
    row=1, col=6
    )


    locationcounts = locationcount.nlargest(10, "Location_Count")
    toploc = locationcounts["Top_Locations"].tolist()
    toploc_c = locationcounts["Location_Count"].tolist()

    topthresl = topdeviceL["Threshold_Label"].tolist()
    topthresl_c = topdeviceL["Label_Count"].tolist()

    toprefM = topdeviceM["Reference_Measure"].tolist()
    toprefM_c = topdeviceM["Measure_Count"].tolist()



    fig.add_trace(
    go.Bar(
        x=toprefM,
        y=toprefM_c, 
        name= "Device Reference Measure",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=2, col=4
    )

    fig.add_trace(
    go.Bar(
        x=topthresl,
        y=topthresl_c, 
        name= "Device Threshold Label",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
    )

    fig.add_trace(
    go.Bar(
        x=toploc,
        y=toploc_c, 
        name= "Top Locations ",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=4, col=4
    )


    fig.update_layout(
    height=670,
    template="plotly_dark",
    title = "Alerts (Last Updated: " + str(df_final["ts"].max()) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
    #autosize=False,
    #width=800,
    #height=800,
    geo = dict(
            projection_type="orthographic",
            showcoastlines=True,
            landcolor="white", 
            showland= True,
            showocean = True,
            lonaxis_range= [ 58.219,33.609 ],
            lataxis_range= [38.687,14.625],
            lakecolor="LightBlue"
    ),

    )
    
    #s.enter(10, 1, apicall, (sc,))
    return fig,result,mtm,table1

def do_something(sc): 
    print("Doing stuff...")
    apicall()
    fig,result,mtm,table1 = apicall()
    s.enter(60, 1, do_something, (sc,))


import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly
from dash.dependencies import Input, Output
import dash_table

fig,result,mtm,table1 = apicall()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
    html.Div(style={'width':"80", 'height':"80", 'padding-top':'20px', 'padding-bottom':'20px','backgroundColor': '#111111', 'textAlign': 'center','color': '#7FDBFF'}, 
    children = [
    #html.H1(children='The time is: ' +str(datetime.datetime.now())),
    html.Div(id='live-update-text'),
    dcc.Graph(id='live-update-graph'),
    #html.Br(),
    dcc.Interval(
        id='interval-component',
        interval=1*60000, # in 1 seconds
        n_intervals=0
    ),   
        
    ]),
    html.Div([ 
    html.H4("Alert Information",style={'textAlign': 'center'}),
    dash_table.DataTable(
        id='live-update-table',
        columns=[{"name": i, "id": i} for i in table1.columns],
        data = table1.to_dict('records')
        ),        
    ]
    
    ),
    
])

@app.callback(Output('live-update-text', 'children'),Input('interval-component', 'n_intervals'))

def update_metrics(n):
    time = datetime.now()
    time = time.strftime("%b %d %Y %H:%M")
    style = {'padding': '5px', 'fontSize': '16px'}
    return [
        html.H2(html.Span(f"Current time is {time}")),
    ]
    


# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),Input('interval-component', 'n_intervals'))

def update_graph_live(n):
    
    # Create the graph with subplots
    fig,result,mtm,table1 = apicall()
    fig.update_layout(
        template="plotly_dark",
        title = "Alerts (Last Updated: " + str(result["ts"].max()) + ")",
        #title = "Alerts (Last Updated: " + str(mtm) + ")",
        showlegend=True,
        legend_orientation="h",
        legend=dict(x=0.65, y=0.8),
        geo = dict(
                projection_type="orthographic",
                showcoastlines=True,
                landcolor="white", 
                showland= True,
                showocean = True,
                lakecolor="LightBlue"
        ),

    )
    return fig

#update table
@app.callback(Output('live-update-table', 'data'),Input('interval-component', 'n_intervals'))

def update_table_live(n):
    fig,result,mtm,table1 = apicall()
    
    return table1.to_dict('records')
    
    
if __name__ == '__main__':
    app.run_server()
    s.enter(60, 1, do_something, (s,)),
    s.run()
    